<a href="https://www.kaggle.com/code/shedai/llm-rag-4-dokumantasyon-cevap-robotu?scriptVersionId=249534006" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Geliştirici Verimliliği

Amaç: Bir yazılımcının, API dokümantasyonu hakkında sorduğu bir soruya, ilgili endpoint'in açıklamasını bularak cevap vermek.

In [2]:
# Gerekli kütüphaneler: pip install sentence-transformers scikit-learn numpy
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# --- BİLGİ KAYNAĞI ---
KNOWLEDGE_BASE = """
Endpoint: GET /api/v1/users
Açıklama: Sistemdeki tüm kullanıcıların bir listesini döndürür. Sonuçlar sayfalama ile verilir.
Parametreler: 'limit' (int, bir sayfadaki maksimum kullanıcı sayısı, varsayılan: 20), 'offset' (int, sonuçların başlangıç indeksi, varsayılan: 0).
Başarılı Cevap: 200 OK - Kullanıcı listesini içeren bir JSON array.

Endpoint: GET /api/v1/users/{id}
Açıklama: Belirtilen ID'ye sahip tek bir kullanıcının detaylarını döndürür.
Parametreler: 'id' (string, zorunlu).
Başarılı Cevap: 200 OK - Kullanıcı objesini içeren bir JSON.
Hata Cevabı: 404 Not Found - Belirtilen ID ile bir kullanıcı bulunamazsa.
"""

# --- MODÜLER FONKSİYONLAR ---
def chunk_text(text: str) -> list[str]:
    return [p.strip() for p in text.split('\n\n') if p.strip()]

def get_embeddings(chunks, model):
    return model.encode(chunks)

def find_most_similar_chunks(query, embeddings, chunks, model, top_k=1):
    query_embedding = model.encode([query])
    similarities = cosine_similarity(query_embedding, embeddings)[0]
    top_k_indices = np.argsort(similarities)[-top_k:][::-1]
    return [chunks[i] for i in top_k_indices]

def generate_answer(query, context):
    prompt = f"Dokümantasyon: {context}\n\nSoru: {query}\n\nCevap:"
    print("--- LLM'e Gönderilen Nihai Prompt ---\n", prompt)
    mock_response = f"API dokümantasyonuna göre, '{query}' sorunuzun cevabı şudur: {context}"
    return mock_response

# --- ANA PROGRAM AKIŞI ---
if __name__ == "__main__":
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
    user_query = "Bütün kullanıcıları liste halinde nasıl alabilirim?"
    
    print("--- Geliştirici Verimliliği RAG Demosu ---")
    print(f"Kullanıcı Sorusu: '{user_query}'\n")

    chunks = chunk_text(KNOWLEDGE_BASE)
    print(f"--- 1. Parçalama Sonucu: {len(chunks)} adet API endpoint'i bulundu. ---\n")
    
    document_embeddings = get_embeddings(chunks, embedding_model)
    
    context_chunks = find_most_similar_chunks(user_query, document_embeddings, chunks, embedding_model)
    context_for_llm = "\n".join(context_chunks)
    print("--- 2. Arama Sonucu: En Alakalı Endpoint Bilgisi Bulundu ---\n", context_for_llm, "\n")
    
    final_answer = generate_answer(user_query, context_for_llm)
    print("\n--- 3. Nihai Cevap Üretildi ---")
    print(final_answer)

2025-07-09 05:28:27.248118: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752038907.508174      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752038907.573561      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

--- Geliştirici Verimliliği RAG Demosu ---
Kullanıcı Sorusu: 'Bütün kullanıcıları liste halinde nasıl alabilirim?'

--- 1. Parçalama Sonucu: 2 adet API endpoint'i bulundu. ---



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

--- 2. Arama Sonucu: En Alakalı Endpoint Bilgisi Bulundu ---
 Endpoint: GET /api/v1/users
Açıklama: Sistemdeki tüm kullanıcıların bir listesini döndürür. Sonuçlar sayfalama ile verilir.
Parametreler: 'limit' (int, bir sayfadaki maksimum kullanıcı sayısı, varsayılan: 20), 'offset' (int, sonuçların başlangıç indeksi, varsayılan: 0).
Başarılı Cevap: 200 OK - Kullanıcı listesini içeren bir JSON array. 

--- LLM'e Gönderilen Nihai Prompt ---
 Dokümantasyon: Endpoint: GET /api/v1/users
Açıklama: Sistemdeki tüm kullanıcıların bir listesini döndürür. Sonuçlar sayfalama ile verilir.
Parametreler: 'limit' (int, bir sayfadaki maksimum kullanıcı sayısı, varsayılan: 20), 'offset' (int, sonuçların başlangıç indeksi, varsayılan: 0).
Başarılı Cevap: 200 OK - Kullanıcı listesini içeren bir JSON array.

Soru: Bütün kullanıcıları liste halinde nasıl alabilirim?

Cevap:

--- 3. Nihai Cevap Üretildi ---
API dokümantasyonuna göre, 'Bütün kullanıcıları liste halinde nasıl alabilirim?' sorunuzun cevabı şudur: